In [1]:
import requests
import pandas as pd
import bs4

In [2]:
cols=['country','total_cases','new_cases','total_deaths','new_deaths',\
      'total_recovered','active_cases','serious','cases_per_mpop',\
      'deaths_per_mpop','total_tests','tests_per_mpop','population','continent']

df=pd.DataFrame(columns=cols)
df

,country,total_cases,new_cases,total_deaths,new_deaths,total_recovered,active_cases,serious,cases_per_mpop,deaths_per_mpop,total_tests,tests_per_mpop,population,continent


In [3]:
def convert(s):
    filters=[',','+']
    for filter in filters:
        s=s.replace(filter,'')
    if (s=='' or s==' '): return 0
    else: return s
    
url='https://www.worldometers.info/coronavirus/'
doc=requests.get(url)
soup=bs4.BeautifulSoup(doc.text,'html.parser')

table=soup.find('table',id='main_table_countries_yesterday')
body=table.find('tbody',class_=None)
trs=body.find_all(lambda tag: tag.name=='tr' and not(tag.has_attr('class') and 'total_row_world' in tag['class']))
for tr in trs:
    tds=tr.find_all('td')
    id=int(tds[0].text)-1
    row=[]
    for i in range(1,len(tds)):
        if i==7: continue
        if(tds[i].has_attr('data-continent')):
            row.append(tds[i].text)
            break
        row.append(convert(tds[i].text))
        
    df.loc[id]=row
df

,country,total_cases,new_cases,total_deaths,new_deaths,total_recovered,active_cases,serious,cases_per_mpop,deaths_per_mpop,total_tests,tests_per_mpop,population,continent
0,China,83040,4,4634,0,78341,65,1,58,3,0,0,1439323776,Asia
1,USA,2026493,19044,113055,586,773480,1139958,16907,6124,342,21725064,65657,330885824,North America
2,Brazil,710887,18925,37312,813,325602,347973,8318,3346,176,999836,4706,212467494,South America
3,Russia,476658,8985,5971,112,230688,239999,2300,3266,41,13016023,89193,145930871,Europe
4,Spain,288797,167,27136,0,N/A,N/A,617,6177,580,4465338,95508,46753739,Europe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,Caribbean Netherlands,7,0,,0,7,0,0,267,0,424,16178,26208,North America
211,St. Barth,6,0,,0,6,0,0,608,0,152,15392,9875,North America
212,Lesotho,4,0,,0,2,2,0,2,0,1515,708,2141209,Africa
213,Anguilla,3,0,,0,3,0,0,200,0,41,2734,14995,North America


In [6]:
from datetime import date, timedelta

yesterday=(date.today()-timedelta(1)).strftime('%d%m%Y')

df.to_csv('data_%s.csv'%(yesterday),index=False)